In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [26]:
salaries = []
exchange_rates = []

In [27]:
# Чтение данных из CSV-файлов
salaries = pd.read_csv("C:\Programming\Django\Vacancies.csv")
exchange_rates = pd.read_csv("C:\Programming\Django\Rub.csv", delimiter=',', decimal='.')

C:\Users\vadla\AppData\Local\Temp\ipykernel_4884\3918294947.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  salaries = pd.read_csv("C:\Programming\Django\Vacancies.csv")


In [28]:
# Преобразование столбца 'date' в формат datetime
exchange_rates['date'] = pd.to_datetime(exchange_rates['date'], format='%d.%m.%Y')

In [29]:
# Преобразование столбца 'published_at' в формат datetime
salaries['published_at'] = pd.to_datetime(salaries['published_at'], errors='coerce')

C:\Users\vadla\AppData\Local\Temp\ipykernel_4884\1997534759.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  salaries['published_at'] = pd.to_datetime(salaries['published_at'], errors='coerce')


In [7]:
# Объединение двух датафреймов по дате
merged = pd.merge(salaries, exchange_rates, how='left', left_on=salaries['published_at'].dt.strftime('%Y-%m-%d'), right_on=exchange_rates['date'].dt.strftime('%Y-%m-%d'))

In [10]:
# Преобразование 'salary_from' и 'price' в числа
merged['salary_from'] = pd.to_numeric(merged['salary_from'], errors='coerce')
merged['price'] = pd.to_numeric(merged['price'], errors='coerce')

In [11]:
# Заполнение пропущенных значений в столбце 'salary_from'
merged['salary_from'].fillna(0, inplace=True)

In [13]:
# Конвертация зарплат в рубли, если в вакансии указана валюта
merged['salary_from_rub'] = merged.apply(lambda row: row['salary_from'] * row['price'] if row['salary_currency'] == 'USD' else row['salary_from'], axis=1)

In [14]:
# Удаление строк, где 'salary_from' пропущено
merged = merged.dropna(subset=['salary_from'])

In [15]:
merged['salary_from_rub'] = merged['salary_from_rub'].round()

In [16]:
# Фильтрация вакансий по названию
filtered = merged[merged['name'].str.contains('c\+\+|с\+\+', case=False, na=False)]
# Округление зарплаты до целого числа
filtered['salary_from_rub'] = filtered['salary_from_rub'].round()

C:\Users\vadla\AppData\Local\Temp\ipykernel_4884\1725894402.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['salary_from_rub'] = filtered['salary_from_rub'].round()


In [17]:
# Фильтрация вакансий по названию
filtered = merged[merged['name'].str.contains('c\+\+|с\+\+', case=False, na=False)]

In [19]:
# Группировка по городам и расчет средней зарплаты
grouped = filtered.groupby('area_name')['salary_from_rub'].mean().sort_values(ascending=False)

In [22]:
pd.set_option('display.max_rows', 15)
pd.options.display.float_format = '{:.0f}'.format

In [23]:
print(grouped.to_string())

area_name
Ташкент                                   1110606
Стерлитамак                                165000
Ржев                                       160000
Минеральные Воды                           160000
Лобня                                      160000
Волгодонск                                 143846
Хасавюрт                                   140000
Новочебоксарск                             140000
Березники                                  140000
Находка                                    140000
Кисловодск                                 140000
Черкесск                                   140000
Энгельс                                    140000
Махачкала                                  140000
Орск                                       140000
Салават                                    140000
Балаково                                   140000
Копейск                                    140000
Апрелевка                                  130000
Адлер                                   